**Setup and Dataset Preprocessing**

Imports

In [1]:
from google.colab import drive, files
import copy, os, shutil, time
from zipfile import ZipFile
import torch
from torch import nn, cuda, device, optim
from torch.utils.data import Dataset, random_split, DataLoader
from torchvision import datasets, models, transforms

Dataset directory name and it's intended path

In [2]:
dir_name = "indian_birds_dataset"
dataset_path = os.path.join("/content", dir_name)

Mount Google Drive locally, to access dataset - `archive.zip`.

In [3]:
drive.mount("/content/drive")

Mounted at /content/drive


Extracts the dataset into `"/content"`. **NOTE**: Replace the value `dataset_path` with the path to `archive.zip` in Google Drive (e.g. `"/content/drive/MyDrive/..."`).

In [4]:
dataset_zip_path = "/content/drive/MyDrive/CS614/archive.zip"

with ZipFile(dataset_zip_path, "r") as zip:
  zip.extractall("/content")

Moves the dataset's root directory from `"training_dataset"` to `"indian_birds_dataset"`

In [5]:
shutil.move("/content/training_set", dataset_path)

Delete accidental directory


In [6]:
def delete_dir(path):
  if os.path.isdir(path):
    print(f"Deleting {path}")
    shutil.rmtree(path, ignore_errors=True)

In [7]:
delete_dir(os.path.join(dataset_path, "training_set"))

Static dataset and model variables

In [8]:
input_size = 299
num_classes = 25
batch_size_train = 20
batch_size_vt = 20
num_epochs = 15 
num_workers = 2
learning_rate = 0.0001
momentum = 0.9

dataset_split = {"train" : 0.8, "valid" : 0.1, "test": 0.1}

Model

In [10]:
model = models.resnet34(weights="DEFAULT")
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, num_classes)

Downloading: "https://download.pytorch.org/models/resnet34-b627a593.pth" to /root/.cache/torch/hub/checkpoints/resnet34-b627a593.pth
100%|██████████| 83.3M/83.3M [00:00<00:00, 225MB/s]


Prepares the dataset.

Wrapper class

Because `random_split` doesn't actually split the dataset, but just keeps the indices of the subsets. This class allows me to apply different transformations to training vs the testing and validation subsets. 

In [11]:
class WrapperDataset(Dataset):
    def __init__(self, subset, transform=None):
        self.subset = subset
        self.transform = transform
        
    def __getitem__(self, index):
        x, y = self.subset[index]
        if self.transform:
            x = self.transform(x)
        return x, y
        
    def __len__(self):
        return len(self.subset)

Dataset Preprocessing

In [12]:
full_dataset = datasets.ImageFolder(dataset_path)

class_to_idx = full_dataset.class_to_idx

generator = torch.Generator().manual_seed(42)
_train, _valid, _test = random_split(full_dataset, [x for x in dataset_split.values()], generator=generator)

num_total_samples = len(_train.dataset)
num_samples = {}
for i in range(len(dataset_split)):
  ds = list(dataset_split.keys())[i]
  num_samples[ds] = int(dataset_split[ds] * num_total_samples)

train_transform = transforms.Compose([
    transforms.Resize(input_size),
    transforms.CenterCrop(input_size),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

valid_test_transform = transforms.Compose([
    transforms.Resize(input_size),
    transforms.CenterCrop(input_size),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

train = WrapperDataset(_train, transform=train_transform)
test = WrapperDataset(_test, transform=valid_test_transform)
valid = WrapperDataset(_valid, transform=valid_test_transform)

train_loader = DataLoader(train , batch_size=batch_size_train, num_workers=num_workers,  shuffle=True)
valid_loader = DataLoader(valid, batch_size=batch_size_vt, num_workers=num_workers)
test_loader = DataLoader(test, batch_size=batch_size_vt, num_workers=num_workers)

List of classes and mapping from index to class

In [13]:
classes = list(class_to_idx.keys())

idx_to_class = {}
for clas, index in class_to_idx.items():
  idx_to_class[index] = clas
print(idx_to_class)

{0: 'Asian Green Bee-Eater', 1: 'Brown-Headed Barbet', 2: 'Cattle Egret', 3: 'Common Kingfisher', 4: 'Common Myna', 5: 'Common Rosefinch', 6: 'Common Tailorbird', 7: 'Coppersmith Barbet', 8: 'Forest Wagtail', 9: 'Gray Wagtail', 10: 'Hoopoe', 11: 'House Crow', 12: 'Indian Grey Hornbill', 13: 'Indian Peacock', 14: 'Indian Pitta', 15: 'Indian Roller', 16: 'Jungle Babbler', 17: 'Northern Lapwing', 18: 'Red-Wattled Lapwing', 19: 'Ruddy Shelduck', 20: 'Rufous Treepie', 21: 'Sarus Crane', 22: 'White Wagtail', 23: 'White-Breasted Kingfisher', 24: 'White-Breasted Waterhen'}


GPU/CPU

In [14]:
devic = device("cuda:0" if cuda.is_available() else "cpu")
print("device:", devic)

device: cuda:0


Optimizer

In [15]:
model = model.to(devic)

params = model.parameters()
optimizer = optim.SGD(params, lr=learning_rate, momentum=momentum)

Loss function - cross entropy loss

In [16]:
criterion = nn.CrossEntropyLoss()

---

**Training**

training function

In [17]:
def train(model, optimizer, criterion, device, train_loader, num_epochs, num_samples):

  for epoch in range(num_epochs):
    print('Epoch {}/{}'.format(epoch, num_epochs - 1))
    print('-' * 10)

    model.train()

    current_loss = 0.0

    for inputs, labels in train_loader:
      inputs = inputs.to(device)
      labels = labels.to(device)

      optimizer.zero_grad()

      with torch.set_grad_enabled(True):
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        
        loss.backward()
        optimizer.step()

      current_loss += loss.item() * inputs.size(0)

    epoch_loss = current_loss / num_samples["train"]
        
    print('epoch_loss: {:.4f}'.format(epoch_loss))

Train, NOTE: change `model_path` to where you want to save the model. 

In [18]:
model_path = "/content/model"

start_time = time.time()

train(model, optimizer, criterion, devic, train_loader, num_epochs, num_samples)
torch.save(model.state_dict(), model_path)

time_elapsed = time.time() - start_time
print("time_elapsed", time_elapsed)


Epoch 0/0
----------
Epoch Loss: 2.1081
time_elapsed 577.6148915290833


Downloads the model

In [ ]:
files.download(model_path)